Step 1: import libraries

In [1]:
import numpy as np

In [2]:
def simulate_gbm(s_0,mu,sigma,n_sims,T,N,randomseed =3):
    
    np.random.seed(randomseed)
    
    dt = T/N
    dW = np.random.normal(scale = np.sqrt(dt),size = (n_sims,N))
    W = np.cumsum(dW, axis = 1)
    
    time_step = np.linspace(dt,T,N)
    time_steps = np.broadcast_to(time_step,(n_sims,N))
    S_t = (s_0 * np.exp((mu - 0.5 * sigma ** 2)*time_steps +sigma *W))
    S_t = np.insert(S_t, 0, s_0, axis=1)
    
    return  S_t

In [3]:
def  black_scholes_analytical(S_0, K, T, r, sigma, type='call'):
    d1 = (np.log(S_0 / K)+(r+(0.5*sigma**2)*T))/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if type == "call":
        N_d1 = norm.cdf(d1, 0, 1)
        N_d2 = norm.cdf(d2, 0, 1)
        price = N_d1 * S_0 - N_d2 * K * np.exp(-r*T)
    elif type =="put":
        N_d1 = norm.cdf(-d1, 0, 1)
        N_d2 = norm.cdf(-d2, 0, 1)
        price = K * np.exp(-r * T) * N_d2 - S_0 * N_d1
    else:
        raise ValueError("unknow type")
    return(price)

Step 2: set parameters

In [4]:
S_0 = 36
K = 40
r = 0.06
sigma = 0.2
T = 1 # 1 year
N = 50
dt = T / N
N_SIMS = 10 ** 5
discount_factor = np.exp(-r * dt)
OPTION_TYPE = "put"
POLY_DEGREE = 5

In [5]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, 
                        n_sims=N_SIMS, T=T, N=N)

In [6]:
payoff_matrix = np.maximum(K - gbm_sims, np.zeros_like(gbm_sims))

In [7]:
payoff_matrix

array([[ 4.        ,  2.10159634,  1.60008084, ..., 13.49303775,
        15.10624192, 15.27428876],
       [ 4.        ,  2.92376888,  1.98819159, ...,  0.44185502,
         0.74493406,  0.        ],
       [ 4.        ,  3.06914158,  4.36740774, ...,  2.45247003,
         1.52345511,  0.9529216 ],
       ...,
       [ 4.        ,  3.48864847,  4.00122128, ...,  2.65663333,
         1.97530336,  2.17268717],
       [ 4.        ,  4.66674636,  3.54695151, ...,  2.74204216,
         2.71047385,  1.53941822],
       [ 4.        ,  6.24260565,  7.4774005 , ...,  4.87586212,
         5.74629917,  6.1581424 ]])

In [8]:
value_matrix = np.zeros_like(payoff_matrix)
value_matrix[:, -1] = payoff_matrix[:, -1]

In [9]:
for t in range(N - 1, 0 , -1):
    regression = np.polyfit(gbm_sims[:, t], 
                            value_matrix[:, t + 1] * discount_factor,
                            POLY_DEGREE)
    continuation_value = np.polyval(regression, gbm_sims[:, t])
    value_matrix[:, t] = np.where(
        payoff_matrix[:, t] > continuation_value,
        payoff_matrix[:, t],
        value_matrix[:, t + 1] * discount_factor)

In [10]:
option_premium = np.mean(value_matrix[:, 1] * discount_factor)
option_premium

4.450551012102458

In [11]:
def lsmc_american_option(S_0, K, T, N, r, sigma, n_sims, option_type, poly_degree, random_seed=42):
    
    dt = T / N
    discount_factor = np.exp(-r * dt)
    
    ### use geometric brownian motion to get the share price simulation data
    gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, n_sims=N_SIMS, T=T, N=N)
    
    ###get option final payoff
    if option_type == "call":
        payoff_matrix = np.maximum(gbm_sims - K, np.zeros_like(gbm_sims))
    elif option_type == "put":
        payoff_matrix = np.maximum(K - gbm_sims, np.zeros_like(gbm_sims))

    ###assign value to the expiry option value
    value_matrix = np.zeros_like(payoff_matrix)
    value_matrix[:, -1] = payoff_matrix[:, -1]
    
    ###get LS fitting and use the fitting and relatvie share price to get the option continuation value
    
    for t in range(N - 1, 0 , -1):
        regression = np.polyfit(gbm_sims[:, t], 
                            value_matrix[:, t + 1] * discount_factor,POLY_DEGREE)
        continuation_value = np.polyval(regression, gbm_sims[:, t])
        value_matrix[:, t] = np.where(
            payoff_matrix[:, t] > continuation_value,
            payoff_matrix[:, t],
            value_matrix[:, t + 1] * discount_factor)
    
    option_premium = np.mean(value_matrix[:, 1] * discount_factor)
    return(option_premium)

In [12]:
lsmc_american_option(36, 40, 1, 50, 0.06, 0.2, 50, "put", 5)

4.450551012102458

In [13]:
!pip install quantlib

usig quant lib to price American option

In [14]:
import QuantLib as ql

Quant lib offician MC American option pricing doc

ql.MCAmericanEngine(GeneralizedBlackScholesProcess, traits, timeSteps=None, timeStepsPerYear=None, 
                    antitheticVariate=False, controlVariate=False, requiredSamples=None, 
                    requiredTolerance=None, maxSamples=None, seed=0, polynomOrder=2, 
                    polynomType=0, nCalibrationSamples=2048, 
                    antitheticVariateCalibration=None, seedCalibration=None)


In [15]:
today = ql.Date().todaysDate()
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, ql.Actual365Fixed()))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.01, ql.Actual365Fixed()))
volatility = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), 0.1, ql.Actual365Fixed()))
initialValue = ql.QuoteHandle(ql.SimpleQuote(100))
process = ql.BlackScholesMertonProcess(initialValue, dividendTS, riskFreeTS, volatility)

steps = 200
rng = "pseudorandom" # could use "lowdiscrepancy"
numPaths = 100000

engine = ql.MCAmericanEngine(process, rng, steps, requiredSamples=numPaths)


In [16]:
day_counter = ql.Actual365Fixed()

valuation_date = ql.Date(1, 1, 2020)
expiry_date =  ql.Date(1, 1, 2021)
ql.Settings.instance().evaluationDate = valuation_date

T = day_counter.yearFraction(valuation_date, expiry_date)
print(f'Time to expiry in years: {T}') 

if OPTION_TYPE == 'call':
    option_type_ql = ql.Option.Call
elif OPTION_TYPE == 'put':
    option_type_ql = ql.Option.Put
        
exercise = ql.AmericanExercise(valuation_date, expiry_date)
payoff = ql.PlainVanillaPayoff(option_type_ql, K)

u = ql.SimpleQuote(S_0)
r = ql.SimpleQuote(r)
sigma = ql.SimpleQuote(sigma)

underlying = ql.QuoteHandle(u)
volatility = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), day_counter))
risk_free_rate = ql.YieldTermStructureHandle(
    ql.FlatForward(0, ql.TARGET(), 
                                ql.QuoteHandle(r), 
                                day_counter))

bs_process = ql.BlackScholesProcess(underlying,risk_free_rate,volatility,)

engine = ql.MCAmericanEngine(bs_process, 'PseudoRandom', timeSteps=N, 
                             polynomOrder=POLY_DEGREE, 
                             seedCalibration=42, 
                             requiredSamples=N_SIMS)

option = ql.VanillaOption(payoff, exercise)
option.setPricingEngine(engine)

option_premium_ql = option.NPV()
option_premium_ql



Time to expiry in years: 1.0027397260273974


4.472079660932772